## id: C001

In [2]:
import pandas as pd
import numpy as np
import configparser
import module_table_writer as tw
import module_data_wrangling as dw

question_id = 'C001'
#question_title='How were you made aware of the opportunity to do a PhD at the HPI?'
selected_code = 2 #code that indicates that the question was answered

#Enrolled demographic
enrolled_column_key = 'B003'
enrolled_column_name = 'Enrolled'
enrolled_dictionary = {'0':'Enrolled','B003_01':'Intend to enroll','B003_02':'Intend to re-enroll'}  
#'B003_05':'Do not intend to finish the PhD' -> only two people and answers are mostly empty, so I removed.

#Gender demographic
gender_column_key = 'B004'
gender_column_name = 'Gender'
gender_dictionary = {1:'Female',2:'Male'}
#3:'Non-binary' no one selected this option
#4:'Diverse' only one person selected this option

#-----------------------------------------------------------------
# LOAD CONFIGURATION FILE
config = configparser.ConfigParser()
config.read('config.ini')
project_path = config['file.loading']['project_path']
data_folder = config['file.loading']['data_folder']
latex_tables_folder = config['file.loading']['latex_tables_folder']
question_index_file = config['file.loading']['question_index_file']
data_file = config['file.loading']['data_file']
sep = config['file.loading']['sep']

#----------------------------------------------------------------
#LOAD QUESTION INDEX (Questions id, Question text, Answer alternatives, Details)
question_index_path = f'{project_path}/{data_folder}/{question_index_file}'
#print(question_index_path)
id_data_frame = pd.read_csv(question_index_path,encoding = 'ISO-8859-1')
filtered_df = id_data_frame[id_data_frame['ID']==question_id]
question_title = filtered_df['Question'].iloc[0]

print('ID:'+question_id+', Question Title:' + question_title)

#extract the answer options for this question (it is a column)
options_dict = dw.generate_options_dictionary(filtered_df,'Alternatives')

#----------------------------------------------------------------
#LOAD DATA FILE AND FILTER NECESSARY COLUMNS
file_name_path = f'{project_path}/{data_folder}/{data_file}'
data_frame = pd.read_csv(file_name_path,encoding = 'ISO-8859-1')

my_list = [enrolled_column_key, gender_column_key, question_id]

# Get a list of all keys
options_columns_list = list(options_dict.keys())
options_names_list = list(options_dict.values())

selected_columns = my_list + options_columns_list

# Create a new DataFrame with only the selected columns
data_frame = data_frame[selected_columns]

#----------------------------------------------------------------
#CREATE LATEX TABLE FILE
tables_path = f'{project_path}/{latex_tables_folder}/{question_id[:2]}/'
tables_file_name = f'{question_id}'

ID:C001, Question Title:How were you made aware of the opportunity to do a PhD at the HPI?
{'C001_01': 'Web', 'C001_02': 'Newsletters', 'C001_03': 'Academic conferences', 'C001_04': 'Friends/colleagues', 'C001_05': 'I completed my Bachelor/Master at HPI', 'C001_06': 'Other'}


### Enrollment count and percentages

In [ ]:
group_column_key = enrolled_column_key
dict_column_names = enrolled_dictionary
#ALL
count_df = dw.percentage_options(df_data=data_frame,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
print('Table for '+'All')
count_df = count_df.rename(columns={count_df.columns[0]: 'Answers'})
display(count_df)
latex_table = tw.write_latex_table(count_df,False, 'All', 
                                    question_id, question_title,
                                    column_format='@{}lcc')
tw.table_to_file(latex_table,tables_path,tables_file_name)

# Filter by enrolled
group_categories = dict_column_names.keys()

for group_category in group_categories:
    filtered_df = data_frame[data_frame[group_column_key] == group_category]    
    if(len(filtered_df.index > 0)):#ignores group_categories that are not present 
           
        count_df = dw.percentage_options(df_data = filtered_df,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
        group_name = dict_column_names.get(group_category)
        print('Table for '+group_name)
    
        #Rename columns before printing
        count_df = count_df.rename(columns={count_df.columns[0]: 'Answers'})
        display(count_df)

        latex_table = tw.write_latex_table(count_df,False, group_category, 
                                    question_id, question_title,column_format='@{}lcc')
        tw.table_to_file(latex_table,tables_path,tables_file_name)


### Gender Count and percentages

In [ ]:
group_column_key = gender_column_key
dict_column_names = gender_dictionary
#ALL
count_df = dw.percentage_options(df_data=data_frame,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
print('Table for '+'All')
count_df = count_df.rename(columns={count_df.columns[0]: 'Answers'})
display(count_df)
latex_table = tw.write_latex_table(count_df,False, 'All', 
                                    question_id, question_title,
                                    column_format='@{}lcc')
tw.table_to_file(latex_table,tables_path,tables_file_name)

# Filter by enrolled
group_categories = dict_column_names.keys()

for group_category in group_categories:
    filtered_df = data_frame[data_frame[group_column_key] == group_category]    
    if(len(filtered_df.index > 0)):#ignores group_categories that are not present 
           
        count_df = dw.percentage_options(df_data = filtered_df,
                                options_columns=options_columns_list,
                                options_names=options_names_list,
                                selected_code=selected_code)
        group_name = dict_column_names.get(group_category)
        print('Table for '+group_name)
    
        #Rename columns before printing
        count_df = count_df.rename(columns={count_df.columns[0]: 'Answers'})
        display(count_df)

        latex_table = tw.write_latex_table(count_df,False, group_category, 
                                    question_id, question_title,column_format='@{}lcc')
        tw.table_to_file(latex_table,tables_path,tables_file_name)